In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import glob

### Import and Merge Census Data

In [9]:
def merge_census_dfs(path: str) -> pd.DataFrame:
    """
    Merge multiple census dataframes into a single dataframe based on the given path.
    Args:
        path (str): The path where the census excel files are located.
    Returns:
        pd.DataFrame: The merged dataframe containing the selected features from all census files.
    """
    excel_files = glob.glob(f'{path}/*.xls')
    dfs = []
    for file in excel_files:
        suffix = file.split('\\')[-1].split('.')[0].split('_')[0].upper()
        print(suffix)
 
        df = pd.read_excel(file, decimal=',', na_values='X', engine='calamine')
        
        cols_to_keep = ['Cod_setor'] + [col for col in df.columns if col.startswith('V')]
        df = df[cols_to_keep]

        df = df.rename(columns={col: f'{col}_{suffix}' for col in df.columns if col != 'Cod_setor'})
        dfs.append(df)

    df_merged = dfs[0]
    for df in dfs[1:]:
        df_merged = df_merged.merge(df, on='Cod_setor', how='outer')

    return df_merged


In [11]:
census_path = '../CENSO-2010/EXCEL/'
census = merge_census_dfs(census_path)
census.shape

BASICO
DOMICILIO01
DOMICILIO02
DOMICILIORENDA
ENTORNO01
ENTORNO02
ENTORNO03
ENTORNO04
ENTORNO05
PESSOA01
PESSOA02
PESSOA03
PESSOA04
PESSOA05
PESSOA06
PESSOA07
PESSOA08
PESSOA09
PESSOA10
PESSOA11
PESSOA12
PESSOA13
PESSOARENDA
RESPONSAVEL01
RESPONSAVEL02
RESPONSAVELRENDA


(18363, 4037)

In [12]:
census = census.rename(columns={'Cod_setor': 'CD_GEOCODI'})
census['CD_GEOCODI'] = census['CD_GEOCODI'].astype(str)

In [13]:
cols_to_keep = [
    'CD_GEOCODI', 'V001_ENTORNO01', 'V002_ENTORNO01', 'V003_ENTORNO01',
    'V004_ENTORNO01', 'V001_DOMICILIORENDA', 'V002_DOMICILIORENDA',
    'V003_DOMICILIORENDA', 'V004_DOMICILIORENDA', 'V001_BASICO',
    'V002_BASICO', 'V003_BASICO', 'V004_BASICO', 'V005_BASICO',
    'V006_BASICO', 'V007_BASICO', 'V008_BASICO', 'V009_BASICO',
    'V010_BASICO', 'V011_BASICO', 'V012_BASICO', 'V001_DOMICILIO02',
    'V002_DOMICILIO02', 'V001_DOMICILIO01', 'V002_DOMICILIO01',
    'V001_PESSOA01', 'V086_PESSOA02', 'V001_PESSOA03', 'V002_PESSOA03','V003_PESSOA03',
    'V004_PESSOA03', 'V005_PESSOA03', 'V006_PESSOA03', 'V001_PESSOA12',
    'V001_PESSOA11', 'V001_RESPONSAVEL01', 'V001_RESPONSAVEL02',
]

In [14]:
census = census[cols_to_keep]
census.shape

(18363, 37)

In [15]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18363 entries, 0 to 18362
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CD_GEOCODI           18363 non-null  object 
 1   V001_ENTORNO01       18206 non-null  float64
 2   V002_ENTORNO01       18206 non-null  float64
 3   V003_ENTORNO01       18206 non-null  float64
 4   V004_ENTORNO01       18206 non-null  float64
 5   V001_DOMICILIORENDA  18206 non-null  float64
 6   V002_DOMICILIORENDA  18206 non-null  float64
 7   V003_DOMICILIORENDA  18206 non-null  float64
 8   V004_DOMICILIORENDA  18206 non-null  float64
 9   V001_BASICO          18333 non-null  float64
 10  V002_BASICO          18333 non-null  float64
 11  V003_BASICO          18333 non-null  float64
 12  V004_BASICO          18333 non-null  float64
 13  V005_BASICO          18333 non-null  float64
 14  V006_BASICO          18333 non-null  float64
 15  V007_BASICO          18324 non-null 

In [16]:
# transform int64 columns to float64

int64_cols = census.select_dtypes(include=['int64']).columns
census[int64_cols] = census[int64_cols].astype('float64')
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18363 entries, 0 to 18362
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CD_GEOCODI           18363 non-null  object 
 1   V001_ENTORNO01       18206 non-null  float64
 2   V002_ENTORNO01       18206 non-null  float64
 3   V003_ENTORNO01       18206 non-null  float64
 4   V004_ENTORNO01       18206 non-null  float64
 5   V001_DOMICILIORENDA  18206 non-null  float64
 6   V002_DOMICILIORENDA  18206 non-null  float64
 7   V003_DOMICILIORENDA  18206 non-null  float64
 8   V004_DOMICILIORENDA  18206 non-null  float64
 9   V001_BASICO          18333 non-null  float64
 10  V002_BASICO          18333 non-null  float64
 11  V003_BASICO          18333 non-null  float64
 12  V004_BASICO          18333 non-null  float64
 13  V005_BASICO          18333 non-null  float64
 14  V006_BASICO          18333 non-null  float64
 15  V007_BASICO          18324 non-null 

### Save to File

In [17]:
census.to_parquet('../GENERATED-DATA/census.parquet', index=False)